# Pretreatment of VSL Fault Detection Problem data for ds4f.DeepScanner validation

This is an adaptation of the VSL Fault Detection Dataset for the validation of ds4f.DeepScaner tool, the result of the final thesis DeepScan4Failure.

#### License

```MIT License
Copyright (c) 2023 / HP-SCDS Observatorio 2021-2022 / Máster Data-Science UC /
Diego García Saiz / Javier Alejandro Cuartas Micieces / 2021-2022 / 
DeepScan4Failure

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.



#### Libraries Install and Google Drive Set Up


In [3]:
# from google.colab import drive
# drive.mount("/content/gdrive")
# !ls

<a name="Information"></a>
# __Relevant Information and Background__

Data selected for validation were published by Enet Centre of VSB T.U. Ostrava in a Kaggle competition, several years ago. It is a set of 8.712 time series stored in columns of a parquet format file (800.000 rows of readings each). There is also a csv format file storing indexes and anomaly/non-anomaly labels. Data are voltage signal time series linked to triphasic power lines, so this last csv file is really important to trace each of the time series to groups of 3 that matches to each power line information. ([Enet Centre VSB T.U. Ostrava, 2018](#Bibliography)).

In [4]:
%config completer.use_jedi=False

In [5]:
import os
import pandas as pd

# datapath="/content/gdrive/MyDrive/DeepScan4Failure/data/"
datapath="/home/ubuntu1/cdir/data/"
data_dir =datapath
datap=os.path.join(datapath,"train.parquet")
# We load the file with phase groups, anomaly labels, and the foreign key to
# each time series (column) in the parquet file.  
dfm = pd.read_csv(datapath+"metadata_train.csv")
meta_train_df = pd.read_csv(data_dir + 'metadata_train.csv')
# meta_test_df = pd.read_csv(data_dir + '/metadata_test.csv')

In [6]:
dfpr=pd.read_parquet(datapath+"train.parquet",columns=["0","1","2"])

Some rows (series) have been labeled differently as anomalies or normal cases, for each different phase. For now, I have decided to take them as anomalies, labeling them globally as such. For example:

In [7]:
dfm.loc[dfm["id_measurement"]==2760,:]

,signal_id,id_measurement,phase,target
8280,8280,2760,0,1
8281,8281,2760,1,1
8282,8282,2760,2,0


All measurements addressed through "id_measurement" (three phases groups of readings), have 3 "signal_id" each. It can be checked like this:

In [8]:
import numpy as np
#All measurements addressed through "id_measurement" have 3 "signal_id" each.
np.array([dfm.loc[dfm["id_measurement"]==il,"signal_id"].count() for il in dfm["id_measurement"].unique() if dfm.loc[dfm["id_measurement"]==il,"signal_id"].count()!=3])

array([], dtype=float64)

There are null values in no phase of no observation, as we can check in the following cell, if we remove the comment character (It takes around 25 minutes):

In [9]:
# import time
# st=time.time()
# a=np.array([np.isnan(np.array(pd.read_parquet(datap,columns=[str(el)]))).sum() for el in dfs.columns]).sum() # pandas: reasonable time
# tt=time.time()-st

<a name="Pretreatment"></a>
# __Data Pretreatment__

### Splitting into training and test sets

Splitting into training and validation set: equal proportion of normal and anomaly observations for the validation set is taken. Then, training set is only made of non-anomalous cases.

In [10]:
# In here, anomaly cases (target column equal to 1) are filtered and looped to get
# a set with the ids of the size 3 groups that matches anomalous time series and 
# another set with the ids of randomly selected size 3 groups that matches a
# non-anomalous behaviour. 

idf1=dfm.loc[dfm.loc[:,'target']==1,:]
id1=idf1["id_measurement"].unique().copy()
idf0=dfm.loc[dfm['id_measurement'].apply(lambda x: x not in id1),:]
id0t=idf0['id_measurement'].unique().copy()
id0=np.random.choice(id0t, id1.shape[0], replace=False)

In [11]:
# Both groups (anomalous and non-anomalous sets) are concatenated and randomly shuffled.

id=np.concatenate([id0,id1])
np.random.shuffle(id)

In [12]:
# A new index dataframe, similar to the one of the csv, is made of only those observations
# which index is in the pandas series of the few last cells.

idv=np.array(pd.Series(id).apply(lambda x: dfm.loc[dfm['id_measurement']==x,:].to_dict("records"))).sum()
idxvalidation=pd.DataFrame(idv).loc[:,['id_measurement','signal_id','target']].reset_index(drop=True)
idxvalidation=pd.DataFrame(idv).loc[:,['id_measurement','signal_id','target']].reset_index(drop=True)
idxvalidation.head()

,id_measurement,signal_id,target
0,578,1734,0
1,578,1735,0
2,578,1736,0
3,1594,4782,0
4,1594,4783,0


In [13]:
# A pandas dataframe is built just like the previous cell's one, but this time with
# the training set's 3 phase groups.

idtrain=dfm.loc[(dfm["signal_id"].isin(idxvalidation["signal_id"]).tolist()==np.repeat(False,dfm.shape[0])).tolist(),["id_measurement","signal_id"]].reset_index()
idtrain.head()

,index,id_measurement,signal_id
0,0,0,0
1,1,0,1
2,2,0,2
3,6,2,6
4,7,2,7


In [14]:
# Column single phase indexes are filtered from the previous two dataframes so that
# they become two pandas series with numbers. There must be a number of consecutive
# numbers equal than the number of features (3 in this case according to the 3 phases)
# repeated as many times as groups of measures we have for the algorithm to work 
# properly.

idtrain=idtrain.loc[:,"signal_id"]
idvalidation=idxvalidation.loc[:,"signal_id"]

# Then, anomaly label series is taken from the validation dataframe.

lbvalidation=idxvalidation.loc[:,"target"]

### Normalizing and reducing the size of the dataset

Just as [Kandanaarachchi et al's (2018)](#Bibliography) work suggests, the best normalization method to apply depends both on the dataset and on the anomaly detection method. Nevertheless, It is min-max scaling that works best with most of the datasets and methods they tried, and that's what it has been aplied here. 

### Pretreatment as PD signal (Partial Discharge)

In [15]:
path="/home/ubuntu1/cdir/"

In [17]:
from scipy.optimize import leastsq
import scipy.signal as ssg
import copy
import os
import sys

import matplotlib.pyplot as plt
%matplotlib
import numpy as np
import math
import h5py
import pandas as pd
import scipy.stats
import sklearn
from sklearn import metrics

Using matplotlib backend: QtAgg


#### Functions for pulse extraction

In [18]:
def fit_sinusoid(signal):
    """
    Fits a sinusoid to the signal.
    
    Source: https://www.kaggle.com/code/jeffreyegan/vsb-power-line-fault-detection-approach
    Creator: Jeffrey Egan
    Modifications: Javier Cuartas
    """
    t = np.linspace(0, 2*np.pi, len(signal))  # data covers one period
    guess_mean = np.mean(signal)
    guess_std = 3*np.std(signal)/(2**0.5)/(2**0.5)
    guess_phase = 0
    guess_freq = 1
    guess_amp = 20

    # Define the function to optimize, in this case, we want to minimize the difference
    # between the actual data and our "guessed" parameters
    
    optimize_func = lambda x: x[0]*np.sin(x[1]*t+x[2]) + x[3] - signal
    est_amp, est_freq, est_phase, est_mean = leastsq(optimize_func, [guess_amp, guess_freq, guess_phase, guess_mean])[0]

    signal_fit = est_amp*np.sin(est_freq*t+est_phase) + est_mean

    return signal_fit

def drop_missing(intersect,sample):
    """
    Find intersection of sorted numpy arrays
    
    Since intersect1d sort arrays each time, it's effectively inefficient.
    Here you have to sweep intersection and each sample together to build
    the new intersection, which can be done in linear time, maintaining order. 

    Source: https://stackoverflow.com/questions/46572308/intersection-of-sorted-numpy-arrays
    Creator: B. M.
    Modifications: Javier Cuartas
    """
    i=j=k=0
    new_intersect=np.empty_like(intersect)
    while i< intersect.size and j < sample.size:
        if intersect[i]==sample[j]: # the 99% case
            new_intersect[k]=intersect[i]
            k+=1
            i+=1
            j+=1
        elif intersect[i]<sample[j]:
            i+=1
        else : 
            j+=1
    return new_intersect[:k]

def clip(v, l, u):
    """
    Numba helper function to clip a value
    
    Source: https://stackoverflow.com/questions/46572308/intersection-of-sorted-numpy-arrays
    Creator: B. M.
    Modifications: Javier Cuartas
    """
    if v < l:
        v = l
    elif v > u:
        v = u
    return v

def _local_maxima_1d_window_single_pass(x, w):
    """
    Source: https://stackoverflow.com/questions/46572308/intersection-of-sorted-numpy-arrays
    Creator: B. M.
    Modifications: Javier Cuartas
    """
    midpoints = np.empty(x.shape[0] // 2, dtype=np.intp)
    left_edges = np.empty(x.shape[0] // 2, dtype=np.intp)
    right_edges = np.empty(x.shape[0] // 2, dtype=np.intp)
    m = 0  # Pointer to the end of valid area in allocated arrays

    i = 1  # Pointer to current sample, first one can't be maxima
    i_max = x.shape[0] - 1  # Last sample can't be maxima
    while i < i_max:
        # Test if previous sample is smaller
        if x[i - 1] < x[i]:
            i_ahead = i + 1  # Index to look ahead of current sample

            # Find next sample that is unequal to x[i]
            while i_ahead < i_max and x[i_ahead] == x[i]:
                i_ahead += 1
                    
            i_right = i_ahead - 1
            
            f = False
            i_window_end = i_right + w
            while i_ahead < i_max and i_ahead < i_window_end:
                if x[i_ahead] > x[i]:
                    f = True
                    break
                i_ahead += 1
                
            # Maxima is found if next unequal sample is smaller than x[i]
            if x[i_ahead] < x[i]:
                left_edges[m] = i
                right_edges[m] = i_right
                midpoints[m] = (left_edges[m] + right_edges[m]) // 2
                m += 1
                
            # Skip samples that can't be maximum
            i = i_ahead - 1
        i += 1

    # Keep only valid part of array memory.
    midpoints = midpoints[:m]
    left_edges = left_edges[:m]
    right_edges = right_edges[:m]
    
    return midpoints, left_edges, right_edges

def local_maxima_1d_window(x, w=1):
    """
    Find local maxima in a 1D array.
    This function finds all local maxima in a 1D array and returns the indices
    for their midpoints (rounded down for even plateau sizes).
    It is a modified version of scipy.signal._peak_finding_utils._local_maxima_1d
    to include the use of a window to define how many points on each side to use in
    the test for a point being a local maxima.
    Parameters
    ----------
    x : ndarray
        The array to search for local maxima.
    w : np.int
        How many points on each side to use for the comparison to be True
    Returns
    -------
    midpoints : ndarray
        Indices of midpoints of local maxima in `x`.
    Notes
    -----
    - Compared to `argrelmax` this function is significantly faster and can
      detect maxima that are more than one sample wide. However this comes at
      the cost of being only applicable to 1D arrays.
      
    Source: https://stackoverflow.com/questions/46572308/intersection-of-sorted-numpy-arrays
    Creator: B. M.
    Modifications: Javier Cuartas
    """    
        
    fm, fl, fr = _local_maxima_1d_window_single_pass(x, w)
    bm, bl, br = _local_maxima_1d_window_single_pass(x[::-1], w)
    bm = np.abs(bm - x.shape[0] + 1)[::-1]
    bl = np.abs(bl - x.shape[0] + 1)[::-1]
    br = np.abs(br - x.shape[0] + 1)[::-1]

    m = drop_missing(fm, bm)

    return m

def plateau_detection(grad, threshold, plateau_length=5):
    """
    Detect the point when the gradient has reach a plateau.
    
    Source: https://stackoverflow.com/questions/46572308/intersection-of-sorted-numpy-arrays
    Creator: B. M.
    Modifications: Javier Cuartas
    """
    count = 0
    loc = 0
    for i in range(grad.shape[0]):
        if grad[i] > threshold:
            count += 1
        
        if count == plateau_length:
            loc = i - plateau_length
            break
            
    return loc

def get_peaks(
    x, 
    window=25):
    
    """
    Find the peaks in a signal trace.
    Parameters
    ----------
    x : ndarray
        The array to search.
    window : np.int
        How many points on each side to use for the local maxima test
    Returns
    -------
    peaks_x : ndarray
        Indices of midpoints of peaks in `x`.
    peaks_y : ndarray
        Absolute heights of peaks in `x`.
    x_hp : ndarray
        An absolute flattened version of `x`.
        
    Source: https://stackoverflow.com/questions/46572308/intersection-of-sorted-numpy-arrays
    Creator: B. M.
    Modifications: Javier Cuartas
    """

    x_hp = x-ssg.savgol_filter(x, 99, 3)
    x_dn = np.abs(x_hp)

    peaks = local_maxima_1d_window(x_dn, window)

    heights = x_dn[peaks]

    ii = np.argsort(heights)[::-1]

    peaks = peaks[ii]
    heights = heights[ii]

    ky = heights
    kx = np.arange(1, heights.shape[0]+1)
    
    conv_length = 9

    grad = np.diff(ky, 1)/np.diff(kx, 1)
    grad = np.convolve(grad, np.ones(conv_length)/conv_length) if any(grad) else grad#, mode='valid') 
    grad = grad[conv_length-1:-conv_length+1]
    
    knee_x = plateau_detection(grad, -0.01, plateau_length=1000)
    knee_x -= conv_length//2
    
    peaks_x = peaks[:knee_x]
    peaks_y = heights[:knee_x]

    ii = np.argsort(peaks_x)
    peaks_x = peaks_x[ii]
    peaks_y = peaks_y[ii]

    return peaks_x, peaks_y, x_hp


#### Calculate peak features

In [19]:

def calculate_peak_features(px, x_hp0,ruif,wl=25):
    """
    Calculate features for peaks from the provided peaks and signal arrays.
    Parameters
    ----------
    px : ndarray
        Indices of peaks.
    x_hp0 : ndarray
        The array to search.
    ruif : string
        Name for the log file.
    wl : np.int
        How many points on each side to use for large window features
    extwdw:

    Returns
    -------
    features : ndarray
        Features calculate for each peak in `x_hp0`.
        
    Source: https://stackoverflow.com/questions/46572308/intersection-of-sorted-numpy-arrays
    Creator: B. M.
    Changes: Javier Cuartas
    """
    num_peak_features = 34
    
    features = np.empty((px.shape[0], num_peak_features), dtype=np.float64) if px.shape[0]!=0 else np.zeros((1, num_peak_features), dtype=np.float64)
    
    logft = open(path+"loads/load{}.log".format(ruif), "a")
    logft.write("********************************")
    logft.write("\n")

    flagf=0
    if px.shape[0]!=0:
        
      for i in range(px.shape[0]):

          feature_number = 0
          x = px[i]
          h0 = x_hp0[x]
          wl_s = clip(x-wl, 0, 800000-1)
          wl_e = clip(x+wl, 0, 800000-1)
          x_hp_wl0 = x_hp0[wl_s:wl_e+1]*np.sign(h0)
          x_hp_wl0_norm = (x_hp_wl0/np.abs(h0))
          
          sgref=x_hp_wl0_norm[1:len(x_hp_wl0_norm)]-x_hp_wl0_norm[0:len(x_hp_wl0_norm)-1]
          sgref[sgref<0]=-1
          sgref[sgref>0]=1
            
          # First, positions where value is constant are calculated.
          chk0=sgref.copy()
          chk0[sgref==0]=True
          chk0[sgref!=0]=False
          chki=np.logical_xor(np.logical_and(chk0[1:len(chk0)],chk0[0:len(chk0)-1]),chk0[0:len(chk0)-1])

          # Now, positions where trend changes (local maxima/minima) are calculated.
          blk=np.abs(sgref[1:len(sgref)]+sgref[0:len(sgref)-1])
          blki=blk.copy()
          blki[blk==0]=True
          blki[blk!=0]=False
          blki=np.logical_and(np.logical_not(chk0[1:len(chk0)]),blki)

          pksdp=(np.concatenate(np.argwhere(np.logical_or(chki,blki)))+1).tolist()
          logft.write("============================================================")
          logft.write("\n")
          logft.write(str(pksdp))
          logft.write("\n")
          pksd=np.take(x_hp_wl0_norm,pksdp).tolist() if len(pksdp)>0 else []
          pksdr=np.take(x_hp_wl0,pksdp).tolist() if len(pksdp)>0 else []
          if np.std(pksdp)<0:
            break

          prat=np.array(pksd[0:len(pksd)-1])-np.array(pksd[1:len(pksd)]) if len(pksd)>1 else np.max(x_hp_wl0)-np.min(x_hp_wl0)
          pratr=np.array(pksdr[0:len(pksdr)-1])-np.array(pksdr[1:len(pksdr)]) if len(pksdr)>1 else np.max(x_hp_wl0_norm)-np.min(x_hp_wl0_norm)

          #Each peak environment is described by the next set of features:
          #Amount of local minimums/maximums in the peak surroundings:
          features[i, feature_number] = h0
          feature_number += 1
          features[i, feature_number] = len(pksd)
          feature_number += 1
            
          #Amount of local minimums below zero in the peak surroundings:
          features[i, feature_number] = sum(np.array(pksd)<0)
          feature_number += 1 
        
          #Amount of local maximums above zero in the peak surroundings:
          features[i, feature_number] = sum(np.array(pksd)>0)
          feature_number += 1
            
          #Ratio between consecutive differences of local minimum and maximum values:
          features[i, feature_number] = np.mean(prat)
          feature_number += 1
          features[i, feature_number] = np.std(prat)
          feature_number += 1
          features[i, feature_number] = max(prat)
          feature_number += 1
          features[i, feature_number] = min(prat)
          feature_number += 1
          features[i, feature_number] = np.percentile(prat,1)
          feature_number += 1
          features[i, feature_number] = np.percentile(prat,5)
          feature_number += 1
          features[i, feature_number] = np.percentile(prat,25)
          feature_number += 1
          features[i, feature_number] = np.percentile(prat,50)
          feature_number += 1
          features[i, feature_number] = np.percentile(prat,75)
          feature_number += 1
          features[i, feature_number] = np.percentile(prat,95)
          feature_number += 1
          features[i, feature_number] = np.percentile(prat,99)
          feature_number += 1
          features[i, feature_number] = np.mean(pksd)
          feature_number += 1
          features[i, feature_number] = np.std(pksd)
          feature_number += 1
        
          #Peak Height:
          features[i, feature_number] = max(pksd)-min(pksd)
          feature_number += 1
          features[i, feature_number] = np.mean(pratr)
          feature_number += 1
          features[i, feature_number] = np.std(pratr)
          feature_number += 1
          features[i, feature_number] = np.max(pratr)
          feature_number += 1
          features[i, feature_number] = np.min(pratr)
          feature_number += 1
          features[i, feature_number] = np.percentile(pratr,1)
          feature_number += 1
          features[i, feature_number] = np.percentile(pratr,5)
          feature_number += 1
          features[i, feature_number] = np.percentile(pratr,25)
          feature_number += 1
          features[i, feature_number] = np.percentile(pratr,50)
          feature_number += 1
          features[i, feature_number] = np.percentile(pratr,75)
          feature_number += 1
          features[i, feature_number] = np.percentile(pratr,95)
          feature_number += 1
          features[i, feature_number] = np.percentile(pratr,99) 
          feature_number += 1
            
          #Mean of the two portions of the signal:
          features[i, feature_number] = np.mean(pksdr)
          feature_number += 1
          features[i, feature_number] = np.std(pksdr)
          feature_number += 1
        
          #Peak Height, position and standard deviation of positions
          features[i, feature_number] = np.max(x_hp_wl0)-np.min(x_hp_wl0)
          feature_number += 1
          features[i, feature_number] = x
          feature_number += 1
          features[i, feature_number] = np.std(pksdp)
          feature_number += 1
          if i == 0:
            assert feature_number == num_peak_features
      
      features=pd.DataFrame(features,columns=f_names)
      fresultp1=features.quantile(0.01,axis=0).to_numpy()
      fresultp5=features.quantile(0.05,axis=0).to_numpy()
      fresultp25=features.quantile(0.25,axis=0).to_numpy()
      fresultp50=features.quantile(0.50,axis=0).to_numpy()
      fresultp75=features.quantile(0.75,axis=0).to_numpy()
      fresultp95=features.quantile(0.95,axis=0).to_numpy()
      fresultp99=features.quantile(0.99,axis=0).to_numpy()
      fresultm=features.mean(axis=0).to_numpy()
      fresults=features.std(axis=0).to_numpy()
      fresultmin=features.min(axis=0).to_numpy()
      fresultmax=features.max(axis=0).to_numpy()
      fresult=np.concatenate([fresultp1, fresultp5,fresultp25,fresultp50,fresultp75,fresultp95,fresultp99,fresultm,fresults,fresultmin,fresultmax,np.array([len(px),flagf])])

    else:
      flagf=1
      i=0
      logft.write("_no_peaks")
      feature_number=0
      h0 =0
      x_hp_wl0 = x_hp0*np.sign(h0)
      x_hp_wl0_norm = (x_hp_wl0/np.abs(h0))

      pksdp=[]
      pksdr=[]
      pksd=[]
      prat=[]
      pratr=[]
      
      feature_number=0
      features[i, feature_number] = h0
      feature_number += 1
      features[i, feature_number] =  0
      feature_number += 1
      features[i, feature_number] = 0
      feature_number += 1 
      features[i, feature_number] =  0
      feature_number += 1
        
      #Ratio between consecutive differences of local minimum and maximum values:
      features[i, feature_number] =  0
      feature_number += 1
      features[i, feature_number] =  0
      feature_number += 1
      features[i, feature_number] = 0
      feature_number += 1
      features[i, feature_number] = 0
      feature_number += 1
      features[i, feature_number] =  0
      feature_number += 1
      features[i, feature_number] =  0
      feature_number += 1
      features[i, feature_number] = 0
      feature_number += 1
      features[i, feature_number] = 0
      feature_number += 1
      features[i, feature_number] = 0
      feature_number += 1
      features[i, feature_number] = 0
      feature_number += 1
      features[i, feature_number] = 0
      feature_number += 1
      features[i, feature_number] = 0
      feature_number += 1
      features[i, feature_number] = 0
      feature_number += 1
    
      #Peak Height:
      features[i, feature_number] =  0
      feature_number += 1
      features[i, feature_number] = 0
      feature_number += 1
      features[i, feature_number] = 0
      feature_number += 1
      features[i, feature_number] = 0
      feature_number += 1
      features[i, feature_number] = 0
      feature_number += 1
      features[i, feature_number] = 0
      feature_number += 1
      features[i, feature_number] = 0
      feature_number += 1
      features[i, feature_number] = 0
      feature_number += 1
      features[i, feature_number] = 0
      feature_number += 1
      features[i, feature_number] = 0
      feature_number += 1
      features[i, feature_number] =  0
      feature_number += 1
      features[i, feature_number] = 0
      feature_number += 1
        
      #Mean of the two portions of the signal:
      features[i, feature_number] =  0
      feature_number += 1
      features[i, feature_number] = 0
      feature_number += 1
      features[i, feature_number] = np.max(x_hp_wl0)-np.min(x_hp_wl0)
      feature_number += 1
      features[i, feature_number] = len(x_hp_wl0)/2
      feature_number += 1
      features[i, feature_number] = 0
      feature_number += 1
      
      fresult=np.concatenate([np.repeat(features,8),np.repeat([0],num_peak_features),np.repeat(features,2),np.array([0,flagf])])
 
    logft.close()
    return fresult

def process_signal(
    data,ruif,wl,
    window=25
  ):
    """
    Process a signal trace to find the peaks and calculate features for each peak.
    Parameters
    ----------
    data : ndarray
        The array to search.
    ruif : string
        Name for the log file.
    wl : np.int
        How many points on each side to use for large window features.
    window : np.int
        How many points on each side to use for the local maxima test.
    Returns
    -------
    f0 : ndarray
        Features calculate for each peak in `data`.
                
    Source: https://stackoverflow.com/questions/46572308/intersection-of-sorted-numpy-arrays
    Creator: B. M.
    Modifications: Javier Cuartas
    """
    
    px0, height0, x_hp0 = get_peaks(
        data.astype(float),
        window=window)
            
    f0 = calculate_peak_features(px0,x_hp0,ruif,wl)
    
    return f0


### Further Exploration

### ***Main Curation Loop - Statistics from time series***

In [22]:
#The following variables store the number of features to get after the loop. There are
#34 features per phase, 11 statistics per earch feature, and 3 phases, as well as 2 additional
#global features.

nwindows=8
nphases=8
wl=25
nfeatures=3
nfeatpphase=34
nfeatstats=11
nfeatglobl=2
nfeatureengineered=1+(nwindows+1)*(nfeatpphase*nfeatstats+nfeatglobl)+nphases*nwindows
 
#n is the iteration number, the id that identifies each of the runs we previously mentioned, 
#and it matches a group of signals stored in a file (a group of features calculated).
#The path of the data source and both destination path and destination filename are set below.

N=6
path=path
parquetfile=path+"data/train.parquet"
   
for n in range(N):
    
    init=n*1500
    endt=1500*(n+1) if (n+1)*1500<8712 else 8712
    name=str(init)+"-"+str(endt)+"r_train_stnw"
    dname=name

    #First, destination files are created (one dataset per signal).

    with h5py.File(path+"data/"+name+".h5", "w") as f:
      f.create_group("general")
      f.flush

    with h5py.File(path+"data/"+name+".h5", "w") as f:
      for j in np.arange(init,endt):
        f.create_dataset("general/"+str(j), shape= (1,nfeatureengineered),dtype="f8")
        f.flush

    #phid will be used to perform computations grouping by phase.

    phid=np.repeat(np.array(list(range(nphases))),pd.read_parquet(parquetfile,columns=[str(0)]).iloc[:,0].shape[0]/nphases)

    #Afterwards, we loop over the set of signals.

    i=0
    ckr=list()
    fts=np.array([])
    for j in np.arange(init,endt):

      flagflat=0
      chkt=list()
      iref=pd.read_parquet(parquetfile,columns=[str(j)]).iloc[:,0].to_numpy()

    #First, a sinusoid is fit for the current signal, and its cuts with x axis are found.

      kref=fit_sinusoid(iref)
      krefm=kref.copy()
      kdel=np.array([])

      lf=0
      cts=0
      d0=0
      d1=0
      kiref=list()
      kuwtref=list()

    #A check of whether the signal belongs to the group of the flat signals is done.

      check=np.std(iref)<3

      if check==True:
        ckr.append(j)

    #If the signal doesn't belong to the group of flat signals we perform some further calculations
    #which are repeated 4 times to be sure about the .

      if check==False:
        for cts in np.arange(0,4):
          
    #Part of the sinusoid fit which was used for previous cts indexes, is removed. The rest is kept 
    #in krefm.
    
    #valj should keep the positions where there is a change in the sign of the sinusoid fit, 
    #d0 and d1 are used to keep the part of the fit which is kept and the influence of this in
    #valj in the following cycle.
    
    #The output of this loop will be a list of positions with the sinusoid fit sign changes in
    #kuwtref and the sinusoid fit sign changes position in kiref.
    
          krefm=np.delete(kref.copy(),kdel) if cts>0 else kref.copy()
          valj=abs(krefm).argmin()
          val=valj+d0+d1
          upwt=-1 if all([val>=len(iref)-3,(kref[val-3]-kref[val])>0]) else 1 if all([val>=len(iref)-3,(kref[val-3]-kref[val])<0]) else 0 if all([val>=len(iref)-3,(kref[val-3]-kref[val])==0]) else -1 if (kref[val]-kref[val+3])>0 else 0 if (kref[val]-kref[val+3])==0 else 1 
    
          if any([(cts==0),(upwt not in kuwtref)]):
            kiref.append(val)
            kuwtref.append(upwt)
            if len(kiref)==2:
              break
            else:
              lf=val-round((1/5)*len(iref)) if max(abs(val-len(iref)),val)==val else 0
              rg=len(iref)-1 if max(abs(val-len(iref)),val)==val else val+round((1/5)*len(iref))-1
              kdel=np.concatenate([kdel,np.concatenate([np.arange(lf,val),np.arange(val,rg+1)])]) if cts>0 else np.concatenate([np.arange(lf,val),np.arange(val,rg+1)])
              d0=len(np.concatenate([np.arange(lf,val),np.arange(val,rg+1)])) if lf==0 else d0
          elif cts<5:
            lf=val-round((1/5)*len(iref)) if max(abs(val-len(iref)),val)==val else 0
            rg=len(iref)-1 if max(abs(val-len(iref)),val)==val else val+round((1/5)*len(iref))-1
            d1=len(np.concatenate([np.arange(lf,val),np.arange(val,rg+1)])) if lf==0 else d1
            kdel=np.concatenate([kdel,np.concatenate([np.arange(lf,val),np.arange(val,rg+1)])]) if cts>0 else np.concatenate([np.arange(lf,val),np.arange(val,rg+1)])
          else:
            break

    #Then, sign changes are converted to dataframes so phase per observation is known.
            
        rows=pd.DataFrame({"x":kiref,"s":kuwtref}).sort_values(by='x', ascending=True).reset_index(drop=True)
        del kref, krefm

        zero_ph=np.array([rows.loc[0,"x"] if rows.loc[0,"s"]>0 else rows.loc[1,"x"] if rows.loc[1,"s"]>0 else None]) if check==False else np.array([len(iref)+1])
        ph0=np.concatenate([phid[-rows.loc[0,"x"]:],phid[:len(phid)-rows.loc[0,"x"]]]) if rows.loc[0,"x"]!=0 else phid if check==False else None
        ph1=np.concatenate([phid[-rows.loc[1,"x"]:],phid[:len(phid)-rows.loc[1,"x"]]]) if rows.loc[1,"x"]!=0 else phid if check==False else None
        phj=(ph0 if rows.loc[0,"s"]==1 else ph1 if rows.loc[0,"s"]==-1 else "error") if check==False else np.repeat(0,len(iref))

    #Iterquartile range is calculated so that extreme observations in each signal can be removed.
        
        Q1=pd.Series(iref).quantile(0.25)
        Q3=pd.Series(iref).quantile(0.75)
        bnds=(Q3-Q1)*3

        cnd=((iref<=Q3+bnds) & (iref>=Q1-bnds)).copy()
        iref=iref[cnd]
        phj=phj[cnd]
        
    #Observations are also split into several windows independently of their phase, features are 
    #calculated for each one and stored into fts.
        
        wtdif=np.array([0,len(iref)]) if len(iref)%nwindows==0 else np.array([(len(iref)-nwindows*int(len(iref)/nwindows))/2, len(iref)-(len(iref)-nwindows*int(len(iref)/nwindows))/2]) if (len(iref)-nwindows*int(len(iref)/nwindows))%2==0 else np.array([int((len(iref)-nwindows*int(len(iref)/nwindows))/2),len(iref)-int((len(iref)-nwindows*int(len(iref)/nwindows))/2)-1])
        wdj=np.repeat(np.array(list(range(nwindows))),int(len(iref)/nwindows))
        iref=iref[int(wtdif[0]):int(wtdif[1])]
        phj=phj[int(wtdif[0]):int(wtdif[1])]

        for ik in range(nwindows):
          phr=np.unique(phj[wdj==ik])
          idsph=np.zeros(nphases)    
          idsph[phr]=1
          f0=process_signal(iref[wdj==ik],dname+".log",wl,wl)
          fts=np.concatenate([fts,idsph,f0])

        f0=process_signal(iref,dname+".log",wl,wl)
        fts=np.concatenate([fts,f0,zero_ph])

    #The current set of features are calculated and stored, and the corresponding array is cleaned.

        with h5py.File(path+"data/"+name+".h5", "a") as f:
          dg=f["general/"+str(j)]
          dg[:,:]=fts
          f.flush()
        fts=np.array([])


KeyboardInterrupt: 

In [23]:
nfeatureengineered=nfeatureengineered
f_names=["h0","lenpksd","pksdov0","pksdbe0","meanprat","stdprat","maxprat","minprat","per01prat","per05prat","per25prat","per50prat","per75prat","per95prat","per99prat","meanpksd","stdpksd","max-minpksd","meanpratr","stdpratr","maxpratr","minpratr","per01pratr","per05pratr","per25pratr","per50pratr","per75pratr","per95pratr","per99pratr","meanpksdr","stdpksdr","max-minpksdr","posx","stdpksdp"]
f_ext=[c+el for c in ["per01_","per05_","per25_","per50_","per75_","per95_","per99_","mean_","std_","min_","max_"] for el in f_names ]
f_ads=["lenpx"]
nwindows=8
full=list()
fullw=["wph_"+str(el) for el in np.arange(nwindows)]
fulla=["phase0","target","phase"]
for nw in range(nwindows+1):
  if nw!=nwindows:
    full.append([wel+"_w"+str(nw) for wel in fullw])
  full.append([xt_el+"_w"+str(nw) for xt_el in f_ext])
  full.append([ds_el+"_w"+str(nw) for ds_el in f_ads])
full.append(fulla)
c_names=[f for sl in full for f in sl]


### ***Joining Preprocessed files***

In [ ]:
nfeatures=3
path='/home/ubuntu1/cdir/'

inif=0
endf= meta_train_df.shape[0]
step=1500
dname="data/"
ename="data/curatedset_def.h5"
os.chdir(path+dname)
k=os.listdir()
k.sort()

with h5py.File(path+ename, "w") as f:
  f.create_group("general")
  f.flush
with h5py.File(path+ename, "w") as f:
  for j in np.arange(inif,endf):
    f.create_dataset("general/"+str(j), shape= (1,nfeatureengineered),dtype="f8")
    f.flush

i=0
for r in k:
  print("==================")
  initj=i if i==0 else endtj if endtj<endf else None
  if initj==None: break
  endtj=initj+step if initj+step<endf else endf
  i=i+1
  for j in np.arange(initj,endtj):
    rdg=np.array([])
    with h5py.File(path+dname+r, "r") as f:
      dg=f["general/"+str(j)]
      rdg=dg[:,:]
    with h5py.File(path+ename, "a") as n:
      ndg=n["general/"+str(j)]
      ndg[:,:]=rdg
      n.flush()


### ***Removing flat signals***

In [23]:
lstfullsig=[144,
 145,
 146,
 1740,
 1741,
 1742,
 2571,
 2572,
 2573,
 3564,
 3565,
 3566,
 5643,
 5644,
 5645,
 6396,
 6397,
 6398,
 7905,
 7906,
 7907,
 8691,
 8692,
 8693]

In [24]:
type(idtrain)
print(len(idtrain))
print(idtrain.loc[~idtrain.isin(lstfullsig)].shape)
print(len(idvalidation))
print(idvalidation.loc[~idvalidation.isin(lstfullsig)].shape)

In [25]:
sum(idtrain.isin(np.array(lstfullsig)))

In [26]:
print(len(idtrain))

In [27]:
idtrain=idtrain.loc[~idtrain.isin(lstfullsig)]
f_valref=idvalidation.isin(lstfullsig)
idvalidation=idvalidation.loc[~f_valref]
lbvalidation=lbvalidation.loc[~f_valref]

<a name="Bibliography"></a>
# __Bibliography__/__Webgraphy__

* Addison H, Dane S, Vantuch T. _Power Line Fault Detection_ (2018). Kaggle [Online]. Available: https://www.kaggle.com/competitions/vsb-power-line-fault-detection/data. [Accesed: Apr. 2023].


* Egan J. _VSB Power Line Fault Detection Approach_. [Competition Notebook]. Kaggle: Power Line Fault Detection Competition of  Enet Centre, VSB – TU of Ostrava. Egan J. 2019. 
https://www.kaggle.com/code/jeffreyegan/vsb-power-line-fault-detection-approach [Accessed Sep. 2022]. 


* Mark4h. _VSB_1st_place_solution_. [Competition Notebook]. Kaggle: Power Line Fault Detection Competition of  Enet Centre, VSB – TU of Ostrava. Mark4h. 2019. 
https://www.kaggle.com/code/mark4h/vsb-1st-place-solution [Accessed Sep. 2022].
